NOTE: this is meant to run on labrador (after copying the preprocessed files which are by the way on tolman [see in copy_rs_data.ipynb])

- Just run all of it quickly when there are new subject to normalize the data and concatenate
- By normalization here I mean to per voxel across its time series.

# Import stuff

In [1]:
import os
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import pandas as pd

# Params

In [2]:
#dirs:
pre_proc_dir = 'data/pre_proc_data'
rs_data_path_IDCH_sub_Nums = '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums'
rs_data_path_IDCH_sub_Nums_normed = '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums_normed' # Here each voxel time series will be normed 
rs_data_path_IDCH_sub_Nums_normed_concat = '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums_normed_concat' # Here each voxel time series will be normed and concatenated

# file_format:
any_session_file_format = '_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz'

# R01 to IDCH mapping:
mapping_R01_to_IDCH = {'222': '101', '183': '102', '216': '103', '192': '104', '251': '105', '206': '106', '180': '107', '184': '108', '169': '109', '207': '110',
                       '159': '111', '115': '112', '114': '113', '232': '114', '173': '115', '171': '117', '215': '118', '265': '119', '177': '120', '269': '121',
                       '261': '122'}

mapping_R01_to_IDCH = {'094': '123', '253': '124', '247': '125', '233': '126', '229': '127', '246': '128', '214': '129', '205': '130', '234': '131', '230': '132',
                                        '218': '133', '213': '134', '237': '135', '129': '136', '126': '137', '228': '138', '141': '139', '197': '140', '202': '141', '196': '142',
                                        '181': '143', '189': '144', '142': '145', '176': '148', '163': '149', '099': '152', '052': '153', '063': '154', '082': '155'}

# For next time (still in the middle of participation: '064': '156').

In [3]:
len(mapping_R01_to_IDCH.keys())

29

## Copy data to be with IDCH sub numbers (and extract the brain)

In [8]:
sub_dir

'data/pre_proc_data/PREVIOUS/func'

In [9]:
# get all directories in the pre_proc_dir:
r01_sub_dirs = [f for f in os.listdir(pre_proc_dir) if os.path.isdir(os.path.join(pre_proc_dir, f))]
# iterate over all subjects:
for dir in r01_sub_dirs:
    sub_dir = os.path.join(pre_proc_dir, dir, 'func') 
    if 'PREVIOUS' in sub_dir:
        continue
    sub_R01_id = dir.split('Hab0')[1][:3] # get sub R01 id
    sub_IDCH_id = mapping_R01_to_IDCH[sub_R01_id] # get sub IDCH id

    # get subject's resting state:
    rs_files = [f for f in os.listdir(sub_dir) if any_session_file_format in f]
    for f in rs_files:
        # get the session number:
        session = f.split('rest_run-')[1][0]
        new_file_name = f'sub-{sub_IDCH_id}_ses-{session}_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz'
        # copy the file if not exists:
        if not os.path.exists(os.path.join(rs_data_path_IDCH_sub_Nums, new_file_name)):
            print(f'> Copying {f} to {new_file_name}')
            # os.system(f'cp {os.path.join(sub_dir, f)} {os.path.join(rs_data_path_IDCH_sub_Nums, new_file_name)}')
            os.system(f'fslmaths {os.path.join(sub_dir, f)} -mas $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz {os.path.join(rs_data_path_IDCH_sub_Nums, new_file_name)}')
        # write the file name to the input_files.txt file:


> Copying sub-JODMBMFHab006301_task-rest_run-1_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz to sub-154_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz
> Copying sub-JODMBMFHab006301_task-rest_run-2_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz to sub-154_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz
> Copying sub-JODMBMFHab017601_task-rest_run-1_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz to sub-148_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz
> Copying sub-JODMBMFHab017601_task-rest_run-2_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz to sub-148_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz
> Copying sub-JODMBMFHab016301_ses-1_task-rest_run-1_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz to sub-149_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz
> Copying sub-JODMBMFHab016301_ses-1_task-rest_run-2_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz to sub-149_ses-2_rest_MNI152

# Normalized the data (per voxel across its time series)

In [10]:
rs_data_path_IDCH_sub_Nums_normed

'/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums_normed'

In [13]:
# If normed folder does not exist, create it:
if not os.path.exists(rs_data_path_IDCH_sub_Nums_normed):
    os.makedirs(rs_data_path_IDCH_sub_Nums_normed)

# iterate over all subjects:
file_list = os.listdir(rs_data_path_IDCH_sub_Nums)
file_list.sort()
for file in file_list:
    if 'PREVIOUS' in file:
        continue
    subject = file.split('_')[0].split('-')[1]
    session = file.split('ses-')[1][0]
    ts_normed_file_name = f'sub-{subject}_ses-{session}_rest_MNI152-smoothAROMAnonaggr_bold_brain_normed.nii.gz'
    if not os.path.exists(os.path.join(rs_data_path_IDCH_sub_Nums_normed, ts_normed_file_name)):
        print(f'> Create a normalized version (per voxel across the time series) for {subject} session {session}')
        # calculate the mean and std for each voxel:
        temp_mean_file = os.path.join(rs_data_path_IDCH_sub_Nums_normed, f'{file.split(".")[0]}_mean.nii.gz')
        temp_std_file = os.path.join(rs_data_path_IDCH_sub_Nums_normed, f'{file.split(".")[0]}_std.nii.gz')
        os.system(f'fslmaths {os.path.join(rs_data_path_IDCH_sub_Nums, file)} -Tmean {temp_mean_file}')
        os.system(f'fslmaths {os.path.join(rs_data_path_IDCH_sub_Nums, file)} -Tstd {temp_std_file}')
        # normalize the time series:
        os.system(f'fslmaths {os.path.join(rs_data_path_IDCH_sub_Nums, file)} -sub {temp_mean_file} -div {temp_std_file} {os.path.join(rs_data_path_IDCH_sub_Nums_normed, ts_normed_file_name)}')
        # remove the temp files:
        os.system(f'rm {temp_mean_file}')
        os.system(f'rm {temp_std_file}')


> Create a normalized version (per voxel across the time series) for 123 session 1
> Create a normalized version (per voxel across the time series) for 123 session 2
> Create a normalized version (per voxel across the time series) for 124 session 1
> Create a normalized version (per voxel across the time series) for 124 session 2
> Create a normalized version (per voxel across the time series) for 125 session 1
> Create a normalized version (per voxel across the time series) for 125 session 2
> Create a normalized version (per voxel across the time series) for 126 session 1
> Create a normalized version (per voxel across the time series) for 126 session 2
> Create a normalized version (per voxel across the time series) for 127 session 1
> Create a normalized version (per voxel across the time series) for 127 session 2
> Create a normalized version (per voxel across the time series) for 128 session 1
> Create a normalized version (per voxel across the time series) for 128 session 2
> Cr

# Concatenate the normed ts data from the 2 sessions into one time series

In [16]:
# if normed_concat folder does not exist, create it:
if not os.path.exists(rs_data_path_IDCH_sub_Nums_normed_concat):
    os.makedirs(rs_data_path_IDCH_sub_Nums_normed_concat)

# get all directories in the root directory
normed_sub_list = os.listdir(rs_data_path_IDCH_sub_Nums_normed)
# get the unique subjects:
normed_sub_list = list(set([x.split('_')[0] for x in normed_sub_list]))
normed_sub_list.sort()

for subj in normed_sub_list:
    if 'PREVIOUS' in subj:
        continue
    # get first rest session
    first_rest_session_file = [f for f in os.listdir(rs_data_path_IDCH_sub_Nums_normed) if f'{subj}_ses-1' in f][0]
    # get second rest session
    second_rest_session_file = [f for f in os.listdir(rs_data_path_IDCH_sub_Nums_normed) if f'{subj}_ses-2' in f][0]

    # concatenate session using fslmerge:
    # fslmerge -t <output_file> <input_files>
    if os.path.exists(os.path.join(rs_data_path_IDCH_sub_Nums_normed_concat, f"{subj}_IDCH_id_rs_normed_concat.nii.gz")):
        continue
    print(f'merging data for {subj}')
    os.system(f'fslmerge -t {os.path.join(rs_data_path_IDCH_sub_Nums_normed_concat, f"{subj}_IDCH_id_rs_normed_concat.nii.gz")} {os.path.join(rs_data_path_IDCH_sub_Nums_normed, first_rest_session_file)} {os.path.join(rs_data_path_IDCH_sub_Nums_normed, second_rest_session_file)}')


merging data for sub-123
merging data for sub-124
merging data for sub-125
merging data for sub-126
merging data for sub-127
merging data for sub-128
merging data for sub-129
merging data for sub-130
merging data for sub-131
merging data for sub-132
merging data for sub-133
merging data for sub-134
merging data for sub-135
merging data for sub-136
merging data for sub-137
merging data for sub-138
merging data for sub-139
merging data for sub-140
merging data for sub-141
merging data for sub-142
merging data for sub-143
merging data for sub-144
merging data for sub-145
merging data for sub-148
merging data for sub-149
merging data for sub-152
merging data for sub-153
merging data for sub-154
merging data for sub-155
